In [2]:
import os
import datetime

from transformers import ViTImageProcessor, ViTModel, ViTImageProcessorFast
from PIL import Image
import requests

import pandas as pd
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils.data_loaders import ImageLoader

/Users/shogun/.pyenv/versions/3.10.6/envs/inspiart/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class GoogleVITHuge224Embedding(EmbeddingFunction):


    def __call__(self, input: Documents) -> Embeddings:

        from transformers import ViTImageProcessor, ViTModel, ViTImageProcessorFast

        #Instantiate the image. Convert it to 244 x 244 and normalise RGB between 0 and 1 witha mean of 0.5 for each channel

        self.feature_extractor = ViTImageProcessorFast.from_pretrained('google/vit-huge-patch14-224-in21k')

        #Instantiate the Google ViT with pretrained weights

        self.model = ViTModel.from_pretrained('google/vit-huge-patch14-224-in21k')#Preprocess the data

        inputs = self.feature_extractor(images=input, return_tensors="pt")

        #Embedd the data

        outputs = self.model(**inputs)

        #Convert the embedding to a Numpy array and take the first vector of the Transformer state

        embeddings = outputs.last_hidden_state.data.numpy()[0,0]

        #return the embedding

        return embeddings


chroma_client = chromadb.PersistentClient(path='/Users/shogun/code/gwen-m97/inspiart/raw_data/my_vectordb')
image_loader = ImageLoader()

image_embbeding_function = GoogleVITHuge224Embedding()
images_db = chroma_client.get_or_create_collection(name="images_db", embedding_function=image_embbeding_function, data_loader=image_loader)
query_uris = '/Users/shogun/code/gwen-m97/raw_data/test_images/Two_Young_Girls_at_the_Piano_MET_rl1975.1.201.R.jpg'

image_suggestions = images_db.query(
    query_uris=[query_uris]
)

In [6]:
image_suggestions

{'ids': [['7731',
   '5722',
   '7754',
   '7741',
   '7753',
   '5719',
   '8180',
   '7539',
   '8010',
   '7743']],
 'embeddings': None,
 'documents': [[None, None, None, None, None, None, None, None, None, None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_adoration-of-the-child-1520(2).jpg',
    'image': 'correggio_adoration-of-the-child-1520(2).jpg'},
   {'image': 'cima-da-conegliano_the-deposition.jpg',
    'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/cima-da-conegliano_the-deposition.jpg'},
   {'image': 'correggio_madonna-and-child-1514(1).jpg',
    'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_madonna-and-child-1514(1).jpg'},
   {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_danae-1531(2).jpg',
    'image': 'correggio_danae-1531(2).jpg'},
   {'url': '/Users/shogun/code/gwen-m97/raw_data/raw_image/correggio_ma